In [1]:
from random import randint

def puntiCarta(carta):
    valore = carta%10 + 1
    if valore == 1: return 11
    if valore == 3: return 10
    if valore < 8: return 0
    return valore - 6

def nomeCarta(carta):
    seme = carta/10
    numero = str(carta%10 + 1)
    if seme == 0: return numero + " di denara"
    if seme == 1: return numero + " di spade"
    if seme == 2: return numero + " di bastoni"
    return numero + " di coppe"

def haPresoIlPrimo(self, cartaTirataPerPrima, cartaTirataPerSeconda):
    seme0 = int(cartaTirataPerPrima/10)
    seme1 = int(cartaTirataPerSeconda/10)
    if seme0 == seme1:
        if puntiCarta(cartaTirataPerPrima) > puntiCarta(cartaTirataPerSeconda): return True
        if puntiCarta(cartaTirataPerPrima) < puntiCarta(cartaTirataPerSeconda): return False
        return cartaTirataPerPrima > cartaTirataPerSeconda
    if seme0 == self.briscola: return True
    if seme1 == self.briscola: return False
    return True

In [ ]:
class Mazzo:
    def __init__(self):
        self.carte = []
        for carta in range (0,40):
            self.carte.append(carta)
    
    def carteRimaste(self):
        return len(self.carte)
    
    def pesca(self):
        if self.carteRimaste() == 0:
            return None
        if self.carteRimaste() == 1:
            return self.carte.pop()
        n = randint(0, self.carteRimaste()-1)
        return self.carte.pop(n)
    
class Giocatore:
    def __init__(self):
        self.mano = []
        self.punti = 0
        self.iaTiraPerPrimo = None

    def aggiungiInMano(self, carta):
        self.mano.append(carta)
        self.mano.sort()
    
    def aggiungiPunti(self, punti):
        self.punti += punti

    def tiraPerPrimo(self, statoPartita):
        pass

    def tiraPerSecondo(self, statoPartita, cartaTirataPerPrima):
        pass

In [ ]:
class Environment:
    def reset(self):
        self.mazzo = Mazzo()

        # infos stato generico partita
        self.cartaInFondo = self.mazzo.pesca()
        self.briscola = int(self.cartaInFondo/10)
        self.briscoleUscite = 0

        self.giocatore0 = Giocatore()
        self.giocatore1 = Giocatore()

        turnoDelGiocatore = randint(0,1)
        if turnoDelGiocatore == 0:
            self.giocatoreTiraPerPrimo = self.giocatore0
            self.giocatoreTiraPerSecondo = self.giocatore1
        else:
            self.giocatoreTiraPerPrimo = self.giocatore1
            self.giocatoreTiraPerSecondo = self.giocatore0
        
        for _ in range(3):
            self.fasePescata()

    def step(self): # un turno
        statoPartita = [self.giocatoreTiraPerSecondo.punti, self.cartaInFondo, self.briscoleUscite]
        cartaTirataPerPrima = self.giocatoreTiraPerPrimo.tiraPerPrimo(statoPartita)
        statoPartita = [self.giocatoreTiraPerPrimo.punti, self.cartaInFondo, self.briscoleUscite]
        cartaTirataPerSeconda = self.giocatoreTiraPerSecondo.tiraPerSecondo(statoPartita, cartaTirataPerPrima)

        punti = puntiCarta(cartaTirataPerPrima) + puntiCarta(cartaTirataPerSeconda)
        if haPresoIlPrimo(cartaTirataPerPrima, cartaTirataPerSeconda):
            self.giocatoreTiraPerPrimo.aggiungiPunti(punti)
        else:
            self.giocatoreTiraPerSecondo.aggiungiPunti(punti)
            self.giocatoreTiraPerPrimo, self.giocatoreTiraPerSecondo = self.giocatoreTiraPerSecondo, self.giocatoreTiraPerPrimo

        if self.mazzo.carteRimaste >= 1:
            self.fasePescata()

        return self.partitaFinita()
            
    def fasePescata(self):
        if self.mazzo.carteRimaste > 1:
            pescata = self.mazzo.pesca()
            self.giocatore0.aggiungiInMano(pescata)
            pescata = self.mazzo.pesca()
            self.giocatore1.aggiungiInMano(pescata)
        else:
            pescata = self.mazzo.pesca()
            self.giocatoreTiraPerPrimo.aggiungiInMano(pescata)
            self.giocatoreTiraPerSecondo.aggiungiInMano(self.cartaInFondo)

    def partitaFinita(self):
        return (self.giocatore0.punti>60 or 
                self.giocatore1.punti>60 or 
                (self.giocatore0.punti==60 and self.giocatore1.punti==60))